In [2]:
!curl https://raw.githubusercontent.com/vseloved/prj-nlp-2019/master/tasks/07-language-as-sequence/run-on-test.json --output run-on-test.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  273k  100  273k    0     0   492k      0 --:--:-- --:--:-- --:--:--  492k


In [2]:
import json

with open('run-on-test.json') as f:
    run_on_js = json.load(f)

In [3]:
import random

def rand_run_on_num():
    x = random.random()
    if x < 0.25:
        return 0
    elif 0.25 <= x < 0.97:
        return 1
    
    return 2

In [150]:
from nltk.corpus import brown
from nltk.corpus import reuters
from nltk.corpus import gutenberg

In [5]:
from collections import defaultdict
import nltk

In [271]:
random.seed(273757)

bigrams = defaultdict(float)
trigrams = defaultdict(float)
corpus = []

def tag_tokens(tokens):
    
    tagged = [[token, False] for token in tokens]
    
    #if tagged[-1][0][-1] in '.!?':
    #    tagged[-1] = ['.', False]
    #else:
    #    tagged.append(['.', False])        
    
    return tagged

with open('run-on-corpus.txt', 'w') as f_corpus:
    for fileid in reuters.fileids():
        sents = reuters.sents(fileid)
        run_on_num = rand_run_on_num()
        buff = []
        prev_offset = 0
        file_tokens = []
        for sent in sents[1:]:
            if len(sent) < 2:
                continue
                
            tokens = tag_tokens(sent)
            buff.extend(tokens)
            file_tokens.extend(map(lambda t: t[0].lower(),tokens))
                
            if prev_offset > 0:                
                if buff[prev_offset - 1][0] == '.':
                    del buff[prev_offset - 1]
                    buff[prev_offset - 2][1] = True                
                   
                    if not buff[prev_offset - 1][0].isupper():                
                        lwr = random.random()                
                        if lwr < 0.93:
                            buff[prev_offset - 1][0] = buff[prev_offset -1][0].lower()                                            
            
            prev_offset = len(buff)            
            
            if run_on_num == 0:
                print(buff, file = f_corpus, end = "\n")
                corpus.append(buff)
                run_on_num = rand_run_on_num()
                prev_offset = 0
                buff = []                               
                       
            run_on_num -= 1
     
        for g in nltk.ngrams(file_tokens, 2):        
            bigrams[g] += 1.0
                
        for g in nltk.ngrams(file_tokens, 3):    
            trigrams[g] += 1.0    

In [272]:
def update_freq(ngrams):
    n = sum(ngrams.values())
    for k, v in ngrams.items():
        ngrams[k] = v/n

In [273]:
update_freq(bigrams)
update_freq(trigrams)

In [274]:
sorted(bigrams.items(), key=lambda kv: kv[1], reverse = True)

[(("'", 's'), 0.006260742188318365),
 (('.', 'the'), 0.005543409125381607),
 (('said', '.'), 0.005032225517447702),
 (('of', 'the'), 0.004968537067934625),
 (('in', 'the'), 0.0047313813940898794),
 (('u', '.'), 0.003455936391999055),
 (('.', 's'), 0.003217942712239663),
 (('s', '.'), 0.0030109552513221637),
 ((',', 'the'), 0.002941400760406567),
 ((',', '000'), 0.0028433540683930145),
 (('said', 'the'), 0.0028400020447344318),
 (('mln', 'dlrs'), 0.0027838556484531666),
 (('.', '"'), 0.0026388806252194527),
 (('the', 'company'), 0.0023656906970449397),
 (('he', 'said'), 0.002091662762955781),
 (('1', '.'), 0.0020573045204553054),
 (('for', 'the'), 0.0019374696746609639),
 (('to', 'the'), 0.0018318809294155997),
 (('said', 'it'), 0.001627407486242038),
 ((',', 'which'), 0.0014707003802032835),
 (('on', 'the'), 0.0014522642500810771),
 (('the', 'u'), 0.0014480742205078482),
 ((',', 'and'), 0.0013818717532508345),
 (('it', 'said'), 0.0013299153865427983),
 (('.', '5'), 0.001310641250505946

In [275]:
train_index = int(0.7 * len(corpus))

train_data = corpus[: train_index]
test_data = corpus[train_index: ]

In [494]:
import math

def baseline(data):
    result_data = []
    for sent in data:
        result_sent = []
        last_point = 0
        for i, word in enumerate(sent):            
            if (i - last_point) > 3 and i < (len(sent) - 1):
                pbigram = (word[0].lower(), '.')                
                bigram = (word[0].lower(), sent[i+1][0].lower())                                    
                if math.log(bigrams[pbigram] + 0.000000000001) > math.log(bigrams[bigram] + 0.001):    
                    #if not word[1]:
                         #print('false positive', pbigram, bigram, bigrams[pbigram], bigrams[bigram], word, sent[i + 1])
                    result_sent.append([word[0], True])
                    last_point = i
                else:   
                    #if word[1]:
                    #     print('false negative', pbigram, bigram, bigrams[pbigram], bigrams[bigram], word, sent[i + 1])
                    result_sent.append([word[0], False])                                    
            else:
                if word[1]:
                    print('false negative', word, sent[i + 1])
                result_sent.append([word[0], False])
            
        result_data.append(result_sent)   
    
    return result_data

In [495]:
result = baseline(run_on_js)

false negative ['horses', True] ['I', False]
false negative ['it', True] ['my', False]
false negative ['you', True] ['The', False]
false negative ['you', True] ['the', False]
false negative ['cold', True] ['today', False]
false negative ['not', True] ['there', False]
false negative ['you', True] ['I', False]
false negative ['not', True] ['If', False]
false negative ['book', True] ['I', False]
false negative ['hard', True] ['I', False]
false negative ['hypnotic', True] ['she', False]
false negative ['started', True] ['there', False]
false negative ['recording', True] ['We', False]
false negative ['Sam', True] ['have', False]
false negative ['not', True] ['what', False]


In [496]:
def labels_vec(data):
    return [word[1] for sent in data for word in sent]

In [491]:
from sklearn.metrics import classification_report


print(classification_report(labels_vec(run_on_js), labels_vec(result)))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98      4542
        True       0.33      0.01      0.01       155

   micro avg       0.97      0.97      0.97      4697
   macro avg       0.65      0.50      0.50      4697
weighted avg       0.95      0.97      0.95      4697

